In [23]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [24]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [25]:
tf.keras.backend.clear_session()

In [26]:
subject = 'Alagoas - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [27]:
data = pd.read_csv('2003_mo_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Wind Gust (m/s),Year,Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - Consumo de Cimento (t)
0,2003-1,0.635160,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,1.733015,...,2.973119,2002.694978,0.294434,8.765721,30.779440,2.590528e+07,1.303625e+06,7.576443,2.359888e+07,30.109
1,2003-2,0.635571,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,1.732242,...,2.972700,2002.726142,0.296211,8.760196,30.599132,2.593685e+07,1.305020e+06,7.578654,2.360861e+07,27.684
2,2003-3,0.635982,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,1.731790,...,2.972455,2002.744322,0.297521,8.754671,30.545910,2.596843e+07,1.306414e+06,7.580866,2.361833e+07,26.534
3,2003-4,0.636393,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,1.731742,...,2.972429,2002.746270,0.298766,8.749145,30.377624,2.600000e+07,1.307809e+06,7.583078,2.362806e+07,25.109
4,2003-5,0.636804,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,1.752976,...,2.928113,2002.886466,0.299945,8.743620,30.464900,2.603157e+07,1.309203e+06,7.585289,2.363779e+07,25.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.688931,...,2.478485,2021.980331,0.587645,NaN,24.556360,NaN,NaN,NaN,NaN,46.546
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.687139,...,2.609282,2021.986821,0.587084,NaN,24.501314,NaN,NaN,NaN,NaN,47.781
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.805808,...,2.715689,2021.989698,0.585891,NaN,24.609100,NaN,NaN,NaN,NaN,51.171
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.827186,...,2.764618,2021.991370,0.584366,NaN,24.538488,NaN,NaN,NaN,NaN,49.126


In [28]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Relative Humidity (%),Wind Gust (m/s),Year,Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-2.106195,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,0.722841,-0.430431,...,-0.083529,0.169309,-1.703297,-1.400856,-0.877266,-0.885888,-1.644358,-1.297696,-1.913990,-1.857572
1,-2.072296,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,0.720859,-0.428937,...,-0.083110,0.168500,-1.696509,-1.374741,-0.879298,-0.900149,-1.627544,-1.242142,-1.888746,-1.832102
2,-2.038397,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,0.719703,-0.428066,...,-0.082866,0.168029,-1.692550,-1.355481,-0.881330,-0.904358,-1.610731,-1.186588,-1.863502,-1.806633
3,-2.004498,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,0.719579,-0.427972,...,-0.082840,0.167978,-1.692125,-1.337169,-0.883362,-0.917668,-1.593917,-1.131034,-1.838257,-1.781163
4,-1.970599,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,0.774005,-0.682377,...,-0.038523,0.082437,-1.661591,-1.319844,-0.885394,-0.910766,-1.577104,-1.075480,-1.813013,-1.755694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.358381,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.292916,0.886745,...,0.200482,-0.325023,1.625861,0.990315,1.043483,-1.575976,1.179374,-1.254203,1.153065,1.054229
188,1.351367,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.032437,0.448261,...,0.078940,-0.005456,1.627275,1.000562,1.037328,-1.527002,1.172432,-1.239868,1.147901,1.048854
189,1.344353,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.806954,0.199101,...,0.042654,0.034263,1.659336,1.011398,1.031173,-1.472984,1.165489,-1.225534,1.142737,1.043480
190,1.337339,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.281399,-0.079501,...,0.118704,-0.091482,1.695522,1.028853,1.025017,-1.478716,1.158547,-1.211200,1.137574,1.038106


In [29]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      24.166
1      21.774
2      28.228
3      25.525
4      26.207
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Alagoas - Consumo de Cimento (t), Length: 240, dtype: float64

In [30]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Relative Humidity (%),Wind Gust (m/s),Year,Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-2.106195,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,0.722841,-0.430431,...,-0.083529,0.169309,-1.703297,-1.400856,-0.877266,-0.885888,-1.644358,-1.297696,-1.913990,-1.857572
1,-2.072296,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,0.720859,-0.428937,...,-0.083110,0.168500,-1.696509,-1.374741,-0.879298,-0.900149,-1.627544,-1.242142,-1.888746,-1.832102
2,-2.038397,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,0.719703,-0.428066,...,-0.082866,0.168029,-1.692550,-1.355481,-0.881330,-0.904358,-1.610731,-1.186588,-1.863502,-1.806633
3,-2.004498,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,0.719579,-0.427972,...,-0.082840,0.167978,-1.692125,-1.337169,-0.883362,-0.917668,-1.593917,-1.131034,-1.838257,-1.781163
4,-1.970599,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,0.774005,-0.682377,...,-0.038523,0.082437,-1.661591,-1.319844,-0.885394,-0.910766,-1.577104,-1.075480,-1.813013,-1.755694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.429306,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.156307,-0.014717,...,0.335539,-0.742563,1.147629,1.348585,1.320458,-0.588269,1.217771,-1.179058,1.109023,1.141942
158,1.428089,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.210357,0.322148,...,0.470822,-0.973117,1.162701,1.330705,1.307307,-0.687207,1.221600,-1.195069,1.116989,1.144540
159,1.426872,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.053886,0.727376,...,0.522730,-1.109663,1.174060,1.311609,1.294156,-0.785508,1.225429,-1.211080,1.124956,1.147138
160,1.425655,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.258439,1.254783,...,0.530817,-1.220372,1.180367,1.291203,1.281004,-0.883126,1.229259,-1.227092,1.132922,1.149736


In [31]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      24.166
1      21.774
2      28.228
3      25.525
4      26.207
        ...  
157    32.321
158    41.937
159    31.982
160    28.499
161    22.626
Name: Alagoas - Consumo de Cimento (t), Length: 162, dtype: float64

In [32]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [33]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 23)

In [34]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size : split_index + 1]
test_input

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Relative Humidity (%),Wind Gust (m/s),Year,Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
126,0.408114,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.199106,0.982583,...,0.617872,-0.767167,0.534834,0.871197,1.001453,1.498208,0.788737,-0.193817,0.746412,0.852250
127,0.417817,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.118650,0.715926,...,0.541316,-0.526868,0.536859,0.912736,1.019577,1.468126,0.809677,-0.237777,0.767524,0.871358
128,0.427521,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,-0.026306,0.331940,...,0.408315,-0.115473,0.538272,0.960653,1.037701,1.431617,0.830616,-0.281737,0.788636,0.890466
129,0.437224,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.017670,-0.030891,...,0.288209,0.029910,0.570334,1.006010,1.055825,1.391868,0.851555,-0.325697,0.809748,0.909574
130,0.446928,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,-0.219143,-0.445771,...,0.221219,0.004270,0.606519,1.054871,1.073949,1.349013,0.872494,-0.369657,0.830860,0.928682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.358381,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.292916,0.886745,...,0.200482,-0.325023,1.625861,0.990315,1.043483,-1.575976,1.179374,-1.254203,1.153065,1.054229
188,1.351367,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.032437,0.448261,...,0.078940,-0.005456,1.627275,1.000562,1.037328,-1.527002,1.172432,-1.239868,1.147901,1.048854
189,1.344353,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.806954,0.199101,...,0.042654,0.034263,1.659336,1.011398,1.031173,-1.472984,1.165489,-1.225534,1.142737,1.043480
190,1.337339,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.281399,-0.079501,...,0.118704,-0.091482,1.695522,1.028853,1.025017,-1.478716,1.158547,-1.211200,1.137574,1.038106


In [35]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 23)

In [36]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [37]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(18, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')    
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
#                         validation_split=0.15,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [38]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [39]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3564501792, 4121240703, 1051262894, 1785179795, 3699281025, 2727588340, 3268952512, 468954229, 343751643, 3010417764, 3770389402, 2733116924, 2412966278, 745333283, 1987290380, 2408677100, 4049666985, 2501632775, 3613421342, 1169873823]


Step: 0 ___________________________________________
val_loss: 32.66158676147461
winner_seed: 3564501792


Step: 1 ___________________________________________
val_loss: 289.81817626953125


Step: 2 ___________________________________________
val_loss: 289.81817626953125


Step: 3 ___________________________________________
val_loss: 30.083675384521484
winner_seed: 1785179795


Step: 4 ___________________________________________
val_loss: 39.59076690673828


Step: 5 ___________________________________________
val_loss: 289.8182067871094


Step: 6 ___________________________________________
val_loss: 31.830284118652344


Step: 7 ___________________________________________
val_loss: 32.07439422607422


Step: 8 ___________________________________________


In [46]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=3564501792)

Epoch 1/10000
4/4 [==============================] - 2s 114ms/step - loss: 2729.8967 - val_loss: 2740.1125
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 2677.7717 - val_loss: 2682.9595
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 2621.6934 - val_loss: 2621.5498
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 2567.0286 - val_loss: 2561.9666
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 2510.4265 - val_loss: 2511.2820
Epoch 6/10000
4/4 [==============================] - 0s 8ms/step - loss: 2462.0066 - val_loss: 2470.0347
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 2421.5706 - val_loss: 2436.1877
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 2388.6221 - val_loss: 2409.2778
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 2359.7314 - val_loss: 2387.8450
Epoch 10/10000
4/4 [==============================] -

4/4 [==============================] - 0s 8ms/step - loss: 1812.9586 - val_loss: 1867.1556
Epoch 79/10000
4/4 [==============================] - 0s 8ms/step - loss: 1806.9686 - val_loss: 1861.2126
Epoch 80/10000
4/4 [==============================] - 0s 8ms/step - loss: 1801.0161 - val_loss: 1855.2632
Epoch 81/10000
4/4 [==============================] - 0s 8ms/step - loss: 1795.0002 - val_loss: 1849.3854
Epoch 82/10000
4/4 [==============================] - 0s 8ms/step - loss: 1789.1140 - val_loss: 1843.5280
Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 1783.2325 - val_loss: 1837.6827
Epoch 84/10000
4/4 [==============================] - 0s 8ms/step - loss: 1777.3022 - val_loss: 1831.9069
Epoch 85/10000
4/4 [==============================] - 0s 8ms/step - loss: 1771.4117 - val_loss: 1826.1803
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 1765.6925 - val_loss: 1820.3920
Epoch 87/10000
4/4 [==============================] - 0s 8ms/

Epoch 155/10000
4/4 [==============================] - 0s 8ms/step - loss: 1404.2571 - val_loss: 1462.7992
Epoch 156/10000
4/4 [==============================] - 0s 8ms/step - loss: 1399.5868 - val_loss: 1458.1146
Epoch 157/10000
4/4 [==============================] - 0s 8ms/step - loss: 1394.8828 - val_loss: 1453.4312
Epoch 158/10000
4/4 [==============================] - 0s 8ms/step - loss: 1390.1038 - val_loss: 1448.7809
Epoch 159/10000
4/4 [==============================] - 0s 8ms/step - loss: 1385.4901 - val_loss: 1444.1206
Epoch 160/10000
4/4 [==============================] - 0s 8ms/step - loss: 1380.7753 - val_loss: 1439.5000
Epoch 161/10000
4/4 [==============================] - 0s 8ms/step - loss: 1376.1229 - val_loss: 1434.9161
Epoch 162/10000
4/4 [==============================] - 0s 8ms/step - loss: 1371.4713 - val_loss: 1430.4048
Epoch 163/10000
4/4 [==============================] - 0s 8ms/step - loss: 1366.9670 - val_loss: 1425.8884
Epoch 164/10000
4/4 [================

4/4 [==============================] - 0s 8ms/step - loss: 1086.7810 - val_loss: 1148.9330
Epoch 232/10000
4/4 [==============================] - 0s 8ms/step - loss: 1083.0205 - val_loss: 1145.2690
Epoch 233/10000
4/4 [==============================] - 0s 8ms/step - loss: 1079.3411 - val_loss: 1141.6118
Epoch 234/10000
4/4 [==============================] - 0s 8ms/step - loss: 1075.6293 - val_loss: 1137.9762
Epoch 235/10000
4/4 [==============================] - 0s 8ms/step - loss: 1071.9303 - val_loss: 1134.3380
Epoch 236/10000
4/4 [==============================] - 0s 8ms/step - loss: 1068.3701 - val_loss: 1130.6620
Epoch 237/10000
4/4 [==============================] - 0s 8ms/step - loss: 1064.6205 - val_loss: 1127.0421
Epoch 238/10000
4/4 [==============================] - 0s 8ms/step - loss: 1060.8878 - val_loss: 1123.4679
Epoch 239/10000
4/4 [==============================] - 0s 8ms/step - loss: 1057.3297 - val_loss: 1119.8441
Epoch 240/10000
4/4 [==============================] 

4/4 [==============================] - 0s 8ms/step - loss: 832.7125 - val_loss: 898.2465
Epoch 309/10000
4/4 [==============================] - 0s 8ms/step - loss: 829.7859 - val_loss: 895.4398
Epoch 310/10000
4/4 [==============================] - 0s 8ms/step - loss: 827.0056 - val_loss: 892.6213
Epoch 311/10000
4/4 [==============================] - 0s 8ms/step - loss: 824.1238 - val_loss: 889.8431
Epoch 312/10000
4/4 [==============================] - 0s 8ms/step - loss: 821.3231 - val_loss: 887.0944
Epoch 313/10000
4/4 [==============================] - 0s 8ms/step - loss: 818.5151 - val_loss: 884.3408
Epoch 314/10000
4/4 [==============================] - 0s 8ms/step - loss: 815.6821 - val_loss: 881.5801
Epoch 315/10000
4/4 [==============================] - 0s 8ms/step - loss: 812.9118 - val_loss: 878.7906
Epoch 316/10000
4/4 [==============================] - 0s 8ms/step - loss: 810.0884 - val_loss: 876.0065
Epoch 317/10000
4/4 [==============================] - 0s 8ms/step - lo

...

4/4 [==============================] - 0s 8ms/step - loss: 5.4481 - val_loss: 36.7506
Epoch 4073/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.7513 - val_loss: 36.7428
Epoch 4074/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.8820 - val_loss: 36.1620
Epoch 4075/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.7112 - val_loss: 35.7723
Epoch 4076/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.0401 - val_loss: 35.4871
Epoch 4077/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.0641 - val_loss: 35.5473
Epoch 4078/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.8036 - val_loss: 35.0947
Epoch 4079/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.3591 - val_loss: 34.7545
Epoch 4080/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5010 - val_loss: 34.2515
Epoch 4081/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7624 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 6.1851 - val_loss: 37.4243
Epoch 4152/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.3861 - val_loss: 37.6247
Epoch 4153/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.9831 - val_loss: 38.0592
Epoch 4154/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.9070 - val_loss: 37.9795
Epoch 4155/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.1989 - val_loss: 37.8119
Epoch 4156/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.0783 - val_loss: 36.9020
Epoch 4157/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.1534 - val_loss: 35.8789
Epoch 4158/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7709 - val_loss: 35.8286
Epoch 4159/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.6578 - val_loss: 36.0229
Epoch 4160/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7249 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 7.2409 - val_loss: 40.4864
Epoch 4231/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.2506 - val_loss: 39.6091
Epoch 4232/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0018 - val_loss: 38.0476
Epoch 4233/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5369 - val_loss: 37.3184
Epoch 4234/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.0887 - val_loss: 37.0748
Epoch 4235/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.2436 - val_loss: 36.6911
Epoch 4236/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5264 - val_loss: 36.7096
Epoch 4237/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.3726 - val_loss: 37.9266
Epoch 4238/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.3432 - val_loss: 38.4358
Epoch 4239/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.7659 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 6.9497 - val_loss: 36.2123
Epoch 4310/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.2738 - val_loss: 34.8634
Epoch 4311/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.4800 - val_loss: 34.2297
Epoch 4312/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.6565 - val_loss: 34.2479
Epoch 4313/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5247 - val_loss: 34.9357
Epoch 4314/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.9210 - val_loss: 35.8633
Epoch 4315/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.9513 - val_loss: 36.4420
Epoch 4316/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7956 - val_loss: 36.4471
Epoch 4317/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.4874 - val_loss: 36.1787
Epoch 4318/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0681 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 6.4263 - val_loss: 34.9751
Epoch 4389/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.9589 - val_loss: 35.8017
Epoch 4390/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7086 - val_loss: 36.7308
Epoch 4391/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.7857 - val_loss: 37.3520
Epoch 4392/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.6505 - val_loss: 37.5706
Epoch 4393/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.8904 - val_loss: 36.9467
Epoch 4394/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.9144 - val_loss: 36.7716
Epoch 4395/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.6989 - val_loss: 36.6508
Epoch 4395: early stopping


In [47]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [48]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,29.192987,29.101618,28.988585,28.911938,28.933973,32.034889,28.596607,27.429358,27.435108,27.755234,28.165901,28.351398,28.277405,28.336819,28.49638,28.509224,28.439659,28.32373,28.277512,28.268063,28.287243,28.322609,28.319962,28.268969,28.161551,28.590771,38.881111,38.856087,38.887527,38.943008
Target,21.34,26.848,28.074,29.941,28.977,31.58,29.991,26.82,29.446,27.666,22.013,28.025,25.648,31.782,27.768,33.073,30.389,27.657,32.388,30.378,30.641,32.337,36.405,22.499,27.961,32.378,32.582,42.703,38.584,36.503
Error,7.852987,2.253618,0.914585,1.029062,0.043026,0.454889,1.394392,0.609358,2.010891,0.089233,6.152901,0.326399,2.629404,3.445181,0.72838,4.563778,1.949341,0.666731,4.110489,2.109938,2.353758,4.014393,8.085037,5.769968,0.20055,3.787228,6.29911,3.846912,0.303528,2.44001


In [49]:
display(mae)
display(mape)

2.6811693

0.09293974

In [44]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [45]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[314.6767]] - Target[330.72099999999995]| =  Error: [[16.044312]]; MAPE:[[0.04851313]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[-312.97653]] - Target[360.96500000000003]| =  Error: [[673.9415]]; MAPE:[[1.867055]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[-306.82227]] - Target[210.711]| =  Error: [[517.53326]]; MAPE:[[2.4561284]]


[array([[16.044312]], dtype=float32),
 array([[673.9415]], dtype=float32),
 array([[517.53326]], dtype=float32)]

402.50635

1.4572321